In [158]:
import timeit

In [1]:
def permutations(lst):
    if len(lst) == 0:
        return
    elif len(lst) == 1:
        yield lst
    else:
        for i in range(len(lst)):
            rest = lst[:i] + lst[i+1:]
            for p in permutations(rest):
                yield [lst[i]] + p

In [2]:
def _permutations_multi(lst):
    if len(lst) == 0:
        return
    elif len(lst) == 1:
        yield lst
    else:
        for i in range(len(lst)):
            if i == 0 or lst[i] != lst[i-1]:
                rest = lst[:i] + lst[i+1:]
                for p in _permutations_multi(rest):
                    yield [lst[i]] + p

In [3]:
def permutations_multi(counts):
    lst = []
    for val in counts:
        for i in range(counts[val]):
            lst.append(val)
    yield from _permutations_multi(lst)

In [5]:
from server.board import *
from server.board_type import *

In [14]:
def create_boards(board_type, filename):
    output_file = open(filename, "w")
    
    for board_objects in permutations_multi(board_type.num_objects):
        board = Board(board_objects)
        if board.check_constraints(board_type.constraints):
            output_file.write(str(board) + "\n")
            
    output_file.close()

In [16]:
list(permutations_multi({"A": 4, "B": 3, None: 2}))

[['A', 'A', 'A', 'A', 'B', 'B', 'B', None, None],
 ['A', 'A', 'A', 'A', 'B', 'B', None, 'B', None],
 ['A', 'A', 'A', 'A', 'B', 'B', None, None, 'B'],
 ['A', 'A', 'A', 'A', 'B', None, 'B', 'B', None],
 ['A', 'A', 'A', 'A', 'B', None, 'B', None, 'B'],
 ['A', 'A', 'A', 'A', 'B', None, None, 'B', 'B'],
 ['A', 'A', 'A', 'A', None, 'B', 'B', 'B', None],
 ['A', 'A', 'A', 'A', None, 'B', 'B', None, 'B'],
 ['A', 'A', 'A', 'A', None, 'B', None, 'B', 'B'],
 ['A', 'A', 'A', 'A', None, None, 'B', 'B', 'B'],
 ['A', 'A', 'A', 'B', 'A', 'B', 'B', None, None],
 ['A', 'A', 'A', 'B', 'A', 'B', None, 'B', None],
 ['A', 'A', 'A', 'B', 'A', 'B', None, None, 'B'],
 ['A', 'A', 'A', 'B', 'A', None, 'B', 'B', None],
 ['A', 'A', 'A', 'B', 'A', None, 'B', None, 'B'],
 ['A', 'A', 'A', 'B', 'A', None, None, 'B', 'B'],
 ['A', 'A', 'A', 'B', 'B', 'A', 'B', None, None],
 ['A', 'A', 'A', 'B', 'B', 'A', None, 'B', None],
 ['A', 'A', 'A', 'B', 'B', 'A', None, None, 'B'],
 ['A', 'A', 'A', 'B', 'B', 'B', 'A', None, None],


In [18]:
len({"A": 4, "B": 3, None: 2}.keys())

3

In [44]:
def _permutations_no_touch(lst):
    if len(lst) == 0:
        return
    elif len(lst) == 1:
        yield lst
    else:
        for i in range(len(lst)):
            if i == 0 or lst[i] != lst[i-1]:
                rest = lst[:i] + lst[i+1:]
                for p in _permutations_no_touch(rest):
                    if p[0] == lst[i] or p[0] is None or lst[i] is None:
                        yield [lst[i]] + p

def permutations_no_touch(counts, empty):
    if len(counts.keys()) > empty:
        return []
    else:
        lst = []
        for val in counts:
            for i in range(counts[val]):
                lst.append(val)
        for i in range(empty):
            lst.append(None)
        
        for p in _permutations_no_touch(lst):
            if p[0] == p[-1] or p[0] is None or p[-1] is None:
                yield p

In [153]:
def sum_counts(d):
    s = 0
    for k in d:
        s += d[k]
    return s
    
def _fill_no_touch(lst, is_hole):
    if len(lst) == 0:
        if all(is_hole):
            yield [None]*len(is_hole)
        return
    else:
        if is_hole[0]:
            for p in _fill_no_touch(lst, is_hole[1:]):
                yield [None] + p
        else:
            for i in range(len(lst)):
                if i == 0 or lst[i] != lst[i-1]:
                    rest = lst[:i] + lst[i+1:]
                    for p in _fill_no_touch(rest, is_hole[1:]):
                        if len(p) == 0 or p[0] == lst[i] or p[0] is None or lst[i] is None:
                            yield [lst[i]] + p

def fill_no_touch(counts, board):
    is_hole = [obj is not None and board[i-1] is None for i, obj in enumerate(board)]
    is_hole_collapsed = [hole_val for i, hole_val in enumerate(is_hole) if hole_val or board[i] is None]
    holes = sum(is_hole)
    empty = len([obj for obj in board if obj is None]) - sum_counts(counts)
    gaps = holes + empty
    
    if len(counts.keys()) > gaps:
        return []
    else:
        lst = []
        for val in counts:
            for i in range(counts[val]):
                lst.append(val)
        for i in range(empty):
            lst.append(None)
                    
        for p in _fill_no_touch(lst, is_hole_collapsed):
            if p[0] == p[-1] or p[0] is None or p[-1] is None or p[0] not in counts or p[-1] not in counts:
                board_copy = board.copy()
                j = 0
                for i in range(len(board_copy)):
                    if board[i] is None:
                        board_copy[i] = p[j]
                        j += 1
                    elif board[i-1] is None:
                        j += 1
                yield board_copy

In [289]:
def fill_no_touch2(counts, board):
    num_none = len([obj for obj in board if obj is None]) - sum_counts(counts)
    perms = permutations_multi({**counts, None: num_none})
    boards = []
    for p in perms:
        board_copy = board.copy()
        j = 0
        for i in range(len(board)):
            if board[i] is None:
                board_copy[i] = p[j]
                j += 1
        
        if all([board_copy[i] == board_copy[i-1] or board_copy[i] not in counts or board_copy[i-1] not in counts
               for i in range(len(board_copy))]):
            boards.append(board_copy)
    return boards

In [318]:
def sum_counts(d):
    s = 0
    for k in d:
        s += d[k]
    return s
    
def _fill_no_touch(lst, holes, no_touch_objs, t=0):
    if len(lst) == 0:
        if all(holes):
            yield [None]*len(holes)
        return
    else:
        if holes[0]:
            for p in _fill_no_touch(lst, holes[1:], no_touch_objs, t+1):
                can_come_after_hole = len(p) == 0 or p[0] != holes[0][-1]
                if can_come_after_hole:
                    yield [None] + p
        else:
            for i in range(len(lst)):
                is_new_obj = (i == 0 or lst[i] != lst[i-1])
                next_is_hole = len(holes) > 1 and holes[1] != False
                can_come_before_hole = (not next_is_hole) or lst[i] is None or holes[1][0] == lst[i] or holes[1][0] not in no_touch_objs
                if is_new_obj and can_come_before_hole
                    rest = lst[:i] + lst[i+1:]
                    for p in _fill_no_touch(rest, holes[1:], no_touch_objs, t+1):
                        if len(p) == 0 or p[0] == lst[i] or p[0] is None or lst[i] is None:
                            yield [lst[i]] + p

def fill_no_touch(counts, board):
    holes = []
    in_hole = False
    for obj in board:
        if obj is None:
            holes.append(False)
            in_hole = False
        elif in_hole:
            holes[-1] += (obj,)
        else:
            in_hole = True
            holes.append((obj,))
                        
    num_holes = len([val for val in holes if val != False])
    empty = len([obj for obj in board if obj is None]) - sum_counts(counts)
    gaps = num_holes + empty
    
    if len(counts.keys()) > gaps:
        return []
    else:
        lst = []
        for val in counts:
            for i in range(counts[val]):
                lst.append(val)
        for i in range(empty):
            lst.append(None)
                    
        for p in _fill_no_touch(lst, holes, counts.keys()):
            if p[0] == p[-1] or p[0] is None or p[-1] is None or p[0] not in counts or p[-1] not in counts:
                if holes[-1] == False or p[0] not in counts or holes[0] != False or p[0] == holes[-1][-1]:
                    board_copy = board.copy()
                    j = 0
                    for i in range(len(board_copy)):
                        if board[i] is None:
                            board_copy[i] = p[j]
                            j += 1
                        elif board[i-1] is None:
                            j += 1
                    yield board_copy

SyntaxError: invalid syntax (<ipython-input-318-4f235b838c47>, line 23)

In [293]:
list(fill_no_touch({"A": 3, "B": 1}, [None, None, "B", "D", None, None, None, "E", "F", None]))

[['A', None, 'B', 'D', 'A', None, 'B', 'E', 'F', 'A'],
 ['A', None, 'B', 'D', 'B', None, 'A', 'E', 'F', 'A'],
 ['B', None, 'B', 'D', 'A', 'A', 'A', 'E', 'F', None],
 [None, 'B', 'B', 'D', 'A', 'A', 'A', 'E', 'F', None],
 [None, 'B', 'B', 'D', 'A', 'A', None, 'E', 'F', 'A'],
 [None, 'B', 'B', 'D', 'A', None, 'A', 'E', 'F', 'A'],
 [None, 'B', 'B', 'D', None, 'A', 'A', 'E', 'F', 'A'],
 [None, None, 'B', 'D', 'A', 'A', 'A', 'E', 'F', 'B']]

In [227]:
fill_no_touch2({"A": 2, "B": 3}, [None, None, "A", "D", None, None, None, "E", "F", None])

[['A', 'A', 'A', 'D', 'B', 'B', 'B', 'E', 'F', None],
 ['A', None, 'A', 'D', 'B', 'B', 'B', 'E', 'F', 'A'],
 [None, 'A', 'A', 'D', 'B', 'B', 'B', 'E', 'F', 'A']]

In [189]:
def multi_alternate(counts):
    lst = []
    for val in counts:
        for i in range(counts[val]):
            lst.append(val)
    tpl = tuple(lst)
    perms = itertools.permutations(tpl)
    return set(perms)

In [294]:
timeit.timeit('fill_no_touch2({"A": 2, "B": 3}, [None, None, "A", "D", None, None, None, "E", "F", None])', number=10000, globals=globals())

3.301326348002476

In [317]:
timeit.timeit('fill_no_touch({"A": 2, "B": 3}, [None, None, "A", "D", None, None, None, "E", "F", None])', number=10000, globals=globals())

0.0038320259991451167

In [190]:
timeit.timeit("permutations_multi({'A': 3, 'B': 4})", number=100000, globals=globals())

0.03242176799903973

In [167]:
timeit.timeit("multi_alternate({'A': 3, 'B': 4})", number=100000, globals=globals())

26.60144549100005